### 練習問題2.78
scheme-number パッケージの内部⼿続きは、本質的に基本⼿続き +, - その他に対する呼び出しでしかない。<br/>
ここでのタイプタグシステムでは、
各データオブジェクトに型がくっついていないといけないようになっていたので、<br/>
直接この⾔語の基本⼿続きを使うことはできなかった。<br/>
しかし、実際のところ、Lispの実装はすべて型システムを持っていて、内部で使⽤している。<br/>
symbol? や number? のような基本述語は、
データオブジェクトが特定の型を持つか識別している。<br/>
2.4.2 節の type-tag, contents,attach-tag の定義を変更し、<br/>
ここでのジェネリックシステムがScheme の内部型システムを利⽤できるようにせよ。<br/>
つまり、システムの挙動はそのままで、
普通の数値を car が scheme-number という記号であるペアとして表すのではなく、<br/>
単に Scheme の数値として表現されるようにせよ。

In [1]:
(define (lookup key table)
  (let ((record (assoc key (cdr table))))
    (if record (cdr record)
        #f)
    )
  )

(define (assoc key records)
  (cond ((null? records) #f)
        ((equal? key (caar records)) (car records))
        (else (assoc key (cdr records)))
        )
  )

(define (insert! key value table)
  (let ((record (assoc key (cdr table))))
    (if record (set-cdr! record value)
        (set-cdr! table (cons (cons key value) (cdr table)))
        )
    )
  'ok)

(define (lookup key-1 key-2 table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (cdr record)
              #f
              )
          )
        #f
        )
    )
  )

(define (insert! key-1 key-2 value table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (set-cdr! record value)
              (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
              )
          )
        (set-cdr! table (cons (list key-1 (cons key-2 value)) (cdr table)))
        )
    )
  'ok)

(define (make-table)
  (let ((local-table (list '*table*)))
    (define (lookup key-1 key-2)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (cdr record) #f))
            #f)
        )
      )
    (define (insert! key-1 key-2 value)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (set-cdr! record value)
                  (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
                  )
              )
            (set-cdr! local-table (cons (list key-1 (cons key-2 value)) (cdr local-table )))
            )
        )
      'ok)
    (define (dispatch m)
      (cond ((eq? m 'lookup-proc) lookup)
            ((eq? m 'insert-proc!) insert!)
            (else (error "Unknown operation: TABLE" m)))
      )
    dispatch)
  )

;(define (make-table) (list '*table*))

(define operation-table (make-table))
(define get (operation-table 'lookup-proc))
(define put (operation-table 'insert-proc!))

; タグつきデータのコンストラクタ
(define (attach-tag type-tag contents)
  (cons type-tag contents))

; タグを返すセレクタ
(define (type-tag datum)
  (if (pair? datum) (car datum)
      (error "Bad tagged datum: TYPE-TAG" datum))
  )
; データを返すセレクタ
(define (contents datum)
  (if (pair? datum) (cdr datum)
      (error "Bad tagged datum: CONTENTS" datum))
  )

;
(define (apply-generic op . args)
  (let ((type-tags (map type-tag args)))
    (let ((proc (get op type-tags)))
      (if proc
          (apply proc (map contents args))
          (error
           "No method for these types: APPLY-GENERIC"
           (list op type-tags ))))))

In [2]:
(define (add x y) (apply-generic 'add x y))
(define (sub x y) (apply-generic 'sub x y))
(define (mul x y) (apply-generic 'mul x y))
(define (div x y) (apply-generic 'div x y))


In [3]:
(define (install-scheme-number-package)
  (define (tag x) (attach-tag 'scheme-number x))
  (put 'add '(scheme-number scheme-number) (lambda (x y) (tag (+ x y))))
  (put 'sub '(scheme-number scheme-number) (lambda (x y) (tag (- x y))))
  (put 'mul '(scheme-number scheme-number) (lambda (x y) (tag (* x y))))
  (put 'div '(scheme-number scheme-number) (lambda (x y) (tag (/ x y))))
  (put 'make 'scheme-number (lambda (x) (tag x)))
'done)

(define (make-scheme-number n) ((get 'make 'scheme-number) n))

(install-scheme-number-package)

done

In [4]:
; タグつきデータのコンストラクタ
(define (attach-tag type-tag contents)
  (if (eq? type-tag 'scheme-number) contents
      (cons type-tag contents))
  )

; タグを返すセレクタ
(define (type-tag datum)
  (if (pair? datum) (car datum)
      'scheme-number)
  )

; データを返すセレクタ
(define (contents datum)
  (if (pair? datum) (cdr datum)
      datum)
  )

In [5]:
(define a (make-scheme-number 2))
(define b (make-scheme-number 4))

(display a)
(display "\n")
(display b)
(display "\n")

2
4


In [6]:
(display (add a b))
(display "\n")
(display (sub a b))
(display "\n")
(display (mul a b))
(display "\n")
(display (div a b))
(display "\n")

6
-2
8
1/2


In [7]:
; number?を使う場合

(define (attach-tag type-tag contents)
  (if (number? contents)
      contents
      (cons type-tag contents)))

(define (type-tag datum)
  (cond ((number? datum) 'scheme-number)
        ((pair? datum) (car datum))
        (else (error "Bad tagged datum: TYPE-TAG" datum))))

(define (contents datum)
  (cond ((number? datum) datum)
        ((pair? datum) (cdr datum))
        (else (error "Bad tagged datum: CONTENTS" datum))))

In [ ]:
; symbol?を使う場合　-> 使ってはできない。

(define (attach-tag type-tag contents)
  (if (number? contents)
      contents
      (cons type-tag contents)))

(define (type-tag datum)
  (cond ((number? datum) 'scheme-number)
        ((pair? datum) (car datum))
        (else (error "Bad tagged datum: TYPE-TAG" datum))))

(define (contents datum)
  (cond ((number? datum) datum)
        ((pair? datum) (cdr datum))
        (else (error "Bad tagged datum: CONTENTS" datum))))